# RewardFunction10
```
Key Elements:
1. Based on RewardFunction_9
2. On straight track, just get rewarded for speed
3. Also rewarded for progress relative to steps
   - Changed target_steps to 300 based on analysis from past testing
4. On curved track, all we care about is lane position
5. Curvature cut-off is 0.97 (same as RewardFunction_6 curvature calculations)
```

In [1]:
def reward_function(params):
    '''
    Curvature calculations are based on K value
    '''
   
    # Read input parameters
    #x_car = params['x']
    #y_car = params['y']
    on_track = params['all_wheels_on_track']
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    progress = params['progress']/100
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    steering = abs(params['steering_angle']) # Only need the absolute steering angle
    speed = params['speed']
    steps = params["steps"]
    
    REWARD_MIN = 0.1
    REWARD_MAX = 100
    TARGET_STEPS = 300
    
    # Calculate 3 marks that are farther and father away from the center line
    marker_1 = 0.25 * track_width
    marker_2 = 0.4 * track_width
    marker_3 = 0.6 * track_width
    
    # Set Base Reward
    if not on_track: # Fail them if off Track
        return REWARD_MIN
    elif progress == 1:
        #the lap is complete.  if we use more steps than TARGET_STEPS, the reward is lower
        return REWARD_MAX * TARGET_STEPS / steps
    elif steps < 3:
        reward = REWARD_MAX
    else:        
        reward = REWARD_MAX * max(progress, 0.4)
    
    # Determine waypoints
    prev_pt = waypoints[closest_waypoints[0]] # closest waypoint behind car
    next_pt = waypoints[closest_waypoints[1]] # closest waypoint ahead of car
    nextnext_pt = waypoints[(closest_waypoints[1] + 1) % len(waypoints)] # waypoint after next_pt
    
    prev_next_dist = ((prev_pt[0] - next_pt[0])**2 + (prev_pt[1] - next_pt[1])**2)**0.5    
    next_nextnext_dist = ((next_pt[0] - nextnext_pt[0])**2 + (next_pt[1] - nextnext_pt[1])**2)**0.5   
    prev_nextnext_dist = ((prev_pt[0] - nextnext_pt[0])**2 + (prev_pt[1] - nextnext_pt[1])**2)**0.5
        
    # Find out whether track is straight or curved
    if prev_nextnext_dist/(prev_next_dist + next_nextnext_dist) > .97:
        reward += (speed**2) + (progress*700/(((steps**2) + 1)**.5))
    else:
        #track is curved
        if distance_from_center <= marker_2:
            reward += 5
        elif distance_from_center <= marker_3:
            reward += 3
    
    return float(reward)
